In [1]:
# -------------------------------
#Importing Dependencies
# -------------------------------
import csv
import os
# Organisation Libraries --------
import pandas as pd
import numpy as np

# SQL Libraries
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

import time
from getpass import getpass


In [2]:
# psycopg2
password = getpass()
cn = psycopg2.connect(host = "127.0.0.1", port = "5432", database = "NBA_Analysis", user = "postgres", password = password)
cur = cn.cursor()
cur.execute("Select * FROM nba_data")
query_results = cur.fetchall()

 ········


In [4]:
cur.execute("Select * FROM nba_data LIMIT 0")
colnames = [desc[0] for desc in cur.description]

In [7]:
df = pd.DataFrame(query_results)
df.head()
df.columns = colnames
df.head()

,index,Player_ID,Player,G,GS,MP,FG,FGA,FGPercentage,ThreePM,...,AST,STL,BLK,TOV,PF,PTS,PER,BPM,VORP,Season
0,0,1,Álex Abrines,68,6,15.5,2.0,5.0,0.393,1.4,...,0.6,0.5,0.1,0.5,1.7,6.0,10.1,-1.6,0.1,2016-17
1,1,2,Quincy Acy,38,1,14.7,1.8,4.5,0.412,1.0,...,0.5,0.4,0.4,0.6,1.8,5.8,11.8,-2.1,0.0,2016-17
2,2,3,Steven Adams,80,80,29.9,4.7,8.2,0.571,0.0,...,1.1,1.1,1.0,1.8,2.4,11.3,16.5,-0.2,1.1,2016-17
3,3,4,Arron Afflalo,61,45,25.9,3.0,6.9,0.440,1.0,...,1.3,0.3,0.1,0.7,1.7,8.4,8.9,-3.6,-0.7,2016-17
4,4,5,Alexis Ajinça,39,15,15.0,2.3,4.6,0.500,0.0,...,0.3,0.5,0.6,0.8,2.0,5.3,12.9,-3.3,-0.2,2016-17
